In [34]:
import matplotlib.pyplot as plt
import numpy as np
import random

In [ ]:
%config InlineBackend.figure_format = 'retina'
from qbstyles import mpl_style
mpl_style(dark=True)

In [ ]:
# Load the data from the file
matrix = np.load('correlations.npy')

# cut out the top-left 100x100 section
# matrix = matrix[:100, :100]

# Verify the shape of the matrix
rows, cols = matrix.shape
assert rows == cols

In [ ]:
diag = np.diag(matrix)
self_frequencies = diag
argsorted = np.argsort(self_frequencies)
sort_indices = argsorted[::-1]
sorted_matrix = matrix[sort_indices][:, sort_indices]
sorted_self_frequencies = np.diag(sorted_matrix)

assert np.array_equal(sorted_self_frequencies, diag[sort_indices]), f"first few self-frequencies: {sorted_self_frequencies[:5]}, expected: {diag[sort_indices][:5]}"

print(f"{sort_indices = }")
print(f"diagonal = {np.diag(sorted_matrix)}")

In [ ]:
plt.figure(figsize=(12, 10))
plt.imshow(sorted_matrix, interpolation="none", cmap='inferno')
plt.colorbar(label="occurences")
plt.title("non-zero co-activations")
plt.grid(False)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(12, 10))
plt.imshow(sorted_matrix[:256, :256], interpolation='none', cmap='inferno')
plt.colorbar(label="occurences")
plt.title("non-zero co-activations")
plt.grid(False)
plt.tight_layout()
plt.show()

In [ ]:
# scale each co-activation by the self-frequency of the corresponding neuron
scaled_matrix = np.zeros_like(sorted_matrix, dtype=float)
for i in range(rows):
    for j in range(cols):
        scaled_matrix[i, j] = sorted_matrix[i, j] / np.sqrt(
            self_frequencies[i] * self_frequencies[j]
        )

In [ ]:
plt.figure(figsize=(12, 10))
plt.imshow(scaled_matrix, interpolation="none", cmap='inferno')
plt.colorbar(label="scaled co-activation")
plt.title("scaled co-activations")
plt.grid(False)
plt.tight_layout()
plt.show()

In [ ]:
BLOCK_SIZE = 4

def cost_function_blocks(order: np.ndarray) -> np.int64:
    """Calculate the cost of a visit order."""
    correlation = np.int64(0)
    for i in range(0, len(order) - BLOCK_SIZE):
        correlation += -np.sqrt(
            matrix[order[i], order[i + 1 : i + BLOCK_SIZE]].sum(dtype=np.int64)
        )
    return correlation

def cost_function_fds(order: np.ndarray) -> float:
    """Calculate the cost of a visit order."""
    # Compute all column differences at once
    col_diffs = matrix[np.ix_(order, order[:-1])] - matrix[np.ix_(order, order[1:])]
    return np.sum(np.sqrt(np.absolute(col_diffs)))

def cost_function_r_criterion(order: np.ndarray) -> float:
    """Calculate the R-criterion cost of a visit order."""
    n = len(order)
    # Create distance matrix (how far each position is from diagonal)
    positions = np.arange(n)
    distances = np.abs(positions[:, None] - positions[None, :])
    
    # Reorder the matrix according to the order
    reordered = matrix[np.ix_(order, order)]
    
    # R-criterion: sum of (distance from diagonal * matrix value)
    return np.sum(distances * reordered)

def cost_function_tsp(order: np.ndarray) -> float:
    """Calculate the TSP-like cost of a visit order."""
    n = len(order)
    total_cost = 0.0
    for i in range(n - 1):
        total_cost += -np.sqrt(matrix[order[i], order[i + 1]])
    return total_cost

def cost_function_absence(order: np.ndarray) -> np.int64:
    """Calculate the cost of a visit order."""
    cost = np.int64(0)
    for i in range(BLOCK_SIZE, len(order) - BLOCK_SIZE):
        cost += matrix[order[i], order[i + BLOCK_SIZE:]].sum(dtype=np.int64)
        cost += matrix[order[i], order[:i - BLOCK_SIZE]].sum(dtype=np.int64)
    return cost

cost_function = cost_function_absence

# heuristic 1: swap two random indices
def swap_two(input: np.ndarray) -> np.ndarray:
    """Swap two random indices."""
    # avoid mutating the original order
    order = input.copy()
    assert not order is input
    a, b = random.sample(range(len(order)), 2)
    order[a], order[b] = order[b], order[a]
    return order

# heuristic 2: reverse a random segment
def reverse_segment(input: np.ndarray) -> np.ndarray:
    """Reverse a random segment."""
    # avoid mutating the original order
    order = input.copy()
    assert not order is input
    a, b = sorted(random.sample(range(len(order)), 2))
    order[a : b + 1] = order[a : b + 1][::-1]
    return order

def simulated_annealing(order: np.ndarray, initial_temp=1, cooling_rate=0.99999, min_temp=1e-6) -> tuple[np.ndarray, np.int64]:
    """Simulated annealing to optimise the visit order."""
    current_order = order[:]
    best_order = order[:]
    current_cost = cost_function(order[:])
    best_cost = current_cost
    temp = initial_temp

    while temp > min_temp:
        new_order = random.choice([swap_two, reverse_segment])(current_order)
        new_cost = cost_function(new_order)

        if (
            new_cost < current_cost
            or np.exp((current_cost - new_cost) / temp) > random.random()
        ):
            current_order, current_cost = new_order.copy(), new_cost
            if new_cost < best_cost:
                best_order, best_cost = new_order.copy(), new_cost
                print(f"New best cost: {best_cost}")

        temp *= cooling_rate

    return best_order, best_cost

In [ ]:
# first, try by starting with the 0..2048 indices
initial_order = np.arange(len(matrix))

# final_best_order_1, final_best_cost_1 = simulated_annealing(initial_order)
final_best_order_2, final_best_cost_2 = simulated_annealing(sort_indices)

print(f"cost of default order  : {cost_function(initial_order)}")
print(f"cost of argsort order  : {cost_function(sort_indices)}")
# print(f"cost of best order 1   : {final_best_cost_1}")
print(f"cost of best order 2   : {final_best_cost_2}")

# print(f"before == after for run 1? {np.array_equal(initial_order, final_best_order_1)}")
eq = np.array_equal(sort_indices, final_best_order_2)
print(f"before == after for run 2? {eq}")
# if not eq:
print(f"before: {sort_indices[:10]}")
print(f"after : {final_best_order_2[:10]}")
print(f"cost before: {cost_function(sort_indices)}")
print(f"cost after : {cost_function(final_best_order_2)}")
# 132643157197

In [ ]:
factored_matrix = matrix[final_best_order_2][:, final_best_order_2]
factored_self_freqs = np.diag(factored_matrix)
scaled_factored_matrix = np.zeros_like(factored_matrix, dtype=float)
for i in range(len(factored_matrix)):
    for j in range(len(factored_matrix)):
        scaled_factored_matrix[i, j] = factored_matrix[i, j] / np.sqrt(
            factored_self_freqs[i] * factored_self_freqs[j]
        )

In [ ]:
plt.figure(figsize=(12, 10))
plt.imshow(factored_matrix, interpolation="none", cmap='inferno')
plt.colorbar(label="number of co-activations")
plt.title("factored co-activations")
plt.grid(False)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(12, 10))
plt.imshow(scaled_factored_matrix, interpolation="none", cmap='inferno')
plt.colorbar(label="number of co-activations")
plt.title("factored co-activations")
plt.grid(False)
plt.tight_layout()
plt.show()

In [ ]:
string_0 = ",".join([f"{int(x)}" for x in sort_indices])
# string_1 = ",".join([f"{int(x)}" for x in final_best_order_1])
string_2 = ",".join([f"{int(x)}" for x in final_best_order_2])

# write the orders to a file
with open("visit_order.txt", "w") as f:
    f.write(f"argsort order: {string_0}\n")
    # f.write(f"best order 1 : {string_1}\n")
    f.write(f"best order 2 : {string_2}\n")